### **Sales Analysis**

Total sales in the last quarter

In [ ]:
SELECT 
    SUM(TotalDue) AS TotalSalesLastQuarter
FROM
    SalesLT.SalesOrderHeader
WHERE
    OrderDate >= DATEADD(QUARTER, -1, GETDATE());

Average number of products sold per transaction

In [ ]:
SELECT 
    AVG(OrderQty) AS AvgItemsPerOrder
FROM
    SalesLT.SalesOrderDetail;

Top 5 customers with the highest total purchase value

In [ ]:
SELECT TOP 5 
    c.CustomerID,
    c.FirstName,
    c.LastName,
    SUM(soh.TotalDue) AS TotalSpent
FROM
    SalesLT.Customer c
JOIN
    SalesLT.SalesOrderHeader soh ON c.CustomerID = soh.CustomerID
GROUP BY
    c.CustomerID, c.FirstName, c.LastName
ORDER BY
    TotalSpent DESC;

Month-over-month sales comparison of the 5 best-selling products

In [ ]:
WITH TopProducts AS (
    SELECT TOP 5 
        ProductID,
        SUM(OrderQty) AS TotalSold
    FROM
        SalesLT.SalesOrderDetail
    GROUP BY
        ProductID
    ORDER BY
        TotalSold DESC
),

MonthlySales AS (
    SELECT 
        sod.ProductID,
        FORMAT(soh.OrderDate, 'yyyy-MM') AS Month,
        SUM(sod.OrderQty) AS MonthlyQty
    FROM
        SalesLT.SalesOrderDetail sod
    JOIN
        SalesLT.SalesOrderHeader soh ON sod.SalesOrderID = soh.SalesOrderID
    WHERE
        sod.ProductID IN (SELECT ProductID FROM TopProducts)
    GROUP BY
        sod.ProductID, FORMAT(soh.OrderDate, 'yyyy-MM')
)

SELECT 
    p.ProductID,
    pr.Name AS ProductName,
    ms.Month,
    ms.MonthlyQty
FROM
    MonthlySales ms
JOIN
    SalesLT.Product pr ON ms.ProductID = pr.ProductID
JOIN
    TopProducts p ON ms.ProductID = p.ProductID
ORDER BY
    p.ProductID, ms.Month;

Products with the highest month-over-month sales growth

In [ ]:
WITH MonthlySales AS (
    SELECT 
        ProductID,
        FORMAT(soh.OrderDate, 'yyyy-MM') AS Month,
        SUM(sod.LineTotal) AS MonthlyRevenue
    FROM
        SalesLT.SalesOrderDetail sod
    JOIN
        SalesLT.SalesOrderHeader soh ON sod.SalesOrderID = soh.SalesOrderID
    GROUP BY
        ProductID, FORMAT(soh.OrderDate, 'yyyy-MM')
),
SalesDiff AS (
    SELECT 
        ms1.ProductID,
        ms1.Month AS CurrentMonth,
        ms1.MonthlyRevenue - ISNULL(ms2.MonthlyRevenue, 0) AS RevenueGrowth
    FROM
        MonthlySales ms1
    LEFT JOIN
        MonthlySales ms2 
        ON ms1.ProductID = ms2.ProductID 
        AND DATEFROMPARTS(LEFT(ms1.Month, 4), RIGHT(ms1.Month, 2), 1) = DATEADD(MONTH, 1, DATEFROMPARTS(LEFT(ms2.Month, 4), RIGHT(ms2.Month, 2), 1))
)
SELECT TOP 5 
    ProductID,
    CurrentMonth,
    RevenueGrowth
FROM
    SalesDiff
ORDER BY
    RevenueGrowth DESC;